## Task 3 English language verifcation system.

First of all let's import all necessary libraries.

In [90]:
import os
import numpy as np
import soundfile as sf
import sidekit

from tqdm import tqdm_notebook
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, precision_recall_curve, f1_score, precision_score, recall_score
import xgboost as xgb
from pydub import AudioSegment
from pyAudioAnalysis.pyAudioAnalysis import audioSegmentation as aS

### Next block of code does some basic preprocessing with three provided test files.
### Here I try to solve speaker dearetiztion problem on test files, I try to split every test file on speaker1 and speaker2 segments and save every segment into separate .wav file. I managed to solve this problem with pyAudioAnalysis python library which can be found here: https://github.com/tyiannak/pyAudioAnalysis.

In [24]:
test_files = ['en-en.wav', 'fr-en.wav', 'it-tr.wav']
test_folder = 'test'
for test_file in test_files:
    test_filename = test_file.split('.')[0]
    
    audio = AudioSegment.from_wav(test_file)
    result = aS.speakerDiarization(test_file, 2, lda_dim=0)
    result_diff = np.diff(result)
    splits_times = (np.where(result_diff!=0)[0] + 1) * .2 * 1000
    
    start_time = 0.0 * 1000
    for seg_num, end_time in enumerate(splits_times):
        segment = audio[start_time:end_time]
        segment_file = test_filename + '_{}'.format(seg_num) + '.wav'
        segment.export(os.path.join(test_folder, segment_file), format="wav")
        start_time = end_time
        
    last_segment = audio[start_time:]
    last_segment_file = test_filename + '_{}'.format(seg_num+1) + '.wav'
    last_segment.export(os.path.join(test_folder, last_segment_file), format='wav')

/home/danil/external_apps/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/home/danil/external_apps/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/home/danil/external_apps/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)


### Here we define a function that uses SOX library and processes raw .mp3 or .wav file into .wav file with 8000 Khz samplereate, 16 bit and 1 channel, besides we apply lowpass and highpass filtes to audio in oreder to repeat phone dinamic filters. Also this function allows to cut an audio of arbitrary duration from the initial file. 

In [11]:
def convert(filename, save_path, duration=None):
    
    save_filename = filename.split('/')[-1].split('.')[0]+'.wav'
    tmp_filename = os.path.join(save_path, save_filename)
    
    if duration is None:
    # convert to wav (8khz, 16 bit, 1 channel)
        os.system("sox {} -R --rate 8000 -b 16 -c 1 {} lowpass 3400 highpass 300".format(filename, tmp_filename))
    else:
        os.system("sox {} -R --rate 8000 -b 16 -c 1 {} lowpass 3400 highpass 300 trim 0 {}".format(filename,
                                                                                                   tmp_filename,
                                                                                                   duration))

### Here I define some paths for initial raw data and also several folders for trainig models and it's evaluation. 

    save_path_train_ivectors - path, where all files for training i vectors are hold
    save_path_train_clf      - path, where all files for training final classifier are hold
    save_path_val            - path, where all files for validation are  hold
    save_path_test           - test files path

In [10]:
data_path_eng = 'data_voip_en/data/' # All English audios
data_path_oth = 'Trainingdata/'      # All foreign language audios
data_path_test = 'test/'             # All test audios

save_path_train_ivectors = 'i_vectors/train_ivectors/'
save_path_train_clf = 'i_vectors/train_clf/'
save_path_val = 'i_vectors/validation/'
save_path_test = 'i_vectors/test/'

### Here I split all downloaded data into several subsets:

    train_ivectors: this subset contains 3.7 hours of English speech and 9.2 hours of other languages speech and used for i-vectors ectraction.
    
    train_clf: this subset contains 4.0 hours of English speech and 11.9 hours of other languages speech and used for classifier training.
    
    validation: this subset contains 4.1 hours of English speech and 12.0 hours of other languages speech and used for classifier validation.
    
    test: this subset contains test segments extracted on the previous step.
    
For train_clf and validation sets I've also performed a random clip of audio file to increase the number of short audios for training of classifier. 

In [5]:
np.random.seed(10)
files_eng = np.random.permutation(os.listdir(data_path_eng))
files_oth = np.random.permutation(os.listdir(data_path_oth))
files_test = os.listdir(data_path_test)

files_eng_train_ivectors = files_eng[:int(len(files_eng)*0.15)]
files_eng_train_clf = files_eng[int(len(files_eng)*0.15):int(len(files_eng)*0.35)]
files_eng_val = files_eng[int(len(files_eng)*0.35):int(len(files_eng)*0.55)]

files_oth_train_ivectors = files_oth[:int(len(files_oth)*0.05)]
files_oth_train_clf = files_oth[int(len(files_oth)*0.05):int(len(files_oth)*0.15)]
files_oth_val = files_oth[int(len(files_oth)*0.15):int(len(files_oth)*0.25)]

dict_files = {data_path_eng: {save_path_train_ivectors: files_eng_train_ivectors, 
                              save_path_train_clf:      files_eng_train_clf,
                              save_path_val:            files_eng_val}, 
              data_path_oth: {save_path_train_ivectors: files_oth_train_ivectors, 
                              save_path_train_clf:      files_oth_train_clf,
                              save_path_val:            files_oth_val}}

for data_path in dict_files:
    for save_path in dict_files[data_path]:
        for f in tqdm_notebook(dict_files[data_path][save_path]):
                
            if 'train_clf' in save_path or 'validation' in save_path:
                if f.endswith('mp3'):
                    duration = np.random.uniform(low=3.0, high=10.0)
                else:
                    
                    file = sf.SoundFile(os.path.join(data_path, f))
                    file_duration = len(file) / file.samplerate
                
                    if file_duration > 3:
                        duration = round(np.random.uniform(low=3.0, high=file_duration))
                    else:
                        duration = None
                convert(os.path.join(data_path, f), save_path, duration=duration)
            else:
                convert(os.path.join(data_path, f), save_path)
            
for f in files_test:    
    convert(os.path.join(data_path_test, f), save_path_test)

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

### Here we define the number of distributions for UBM model and the size of i-vectors.

In [6]:
distribNb = 512  # number of Gaussian distributions for the UBM
rank_TV = 400  # Rank of the Total Variability matrix

nbThread = 7

### Now we are ready to extract first 13 MFCC features and save it to a separate folder. 

In [27]:
for extract_folder in [save_path_train_ivectors, save_path_train_clf, save_path_val, save_path_test]:
    
    extract_folder_list = extract_folder.split('/')
    extract_folder_list.insert(1, 'feat')
    feat_folder = '/'.join(extract_folder_list)
    
    extractor = sidekit.FeaturesExtractor(audio_filename_structure=os.path.join(extract_folder, '{}.wav'),
                                          feature_filename_structure=os.path.join(feat_folder, '{}.h5'),
                                          sampling_frequency=8000,
                                          lower_frequency=200,
                                          higher_frequency=3700,
                                          filter_bank="log",
                                          filter_bank_size=24,
                                          window_size=0.025,
                                          shift=0.01,
                                          ceps_number=20,
                                          vad="snr",
                                          snr=40,
                                          pre_emphasis=0.97,
                                          save_param=["vad", "energy", "cep", "fb"],
                                          keep_all_features=True)

    filenames = [f.split('.')[0] for f in os.listdir(extract_folder)]
    channel_list = np.zeros(len(filenames), dtype=np.int8)
    extractor.save_list(show_list=filenames,
                        channel_list=channel_list,
                        num_thread=nbThread)

### Here we define a feature server which helps to maintain UBM model with MFCC features, extracted on a previous step. 

In [28]:
def get_feature_server(feature_filename_structure):
    
    fs = sidekit.FeaturesServer(feature_filename_structure=feature_filename_structure,
                                dataset_list='["cep"]',
                                feat_norm="cmvn",
                                delta=True,
                                double_delta=True,
                                rasta=True,
                                mask='[0-12]',
                                keep_all_features=True)
    
    return fs


fs_train_ivectors = get_feature_server('i_vectors/feat/train_ivectors/{}.h5')
fs_train_clf = get_feature_server('i_vectors/feat/train_clf/{}.h5')
fs_validation = get_feature_server('i_vectors/feat/validation/{}.h5')
fs_test = get_feature_server('i_vectors/feat/test/{}.h5')

ubm_list_train_ivectors = [f.split('.h5')[0] for f in os.listdir('i_vectors/feat/train_ivectors')]
ubm_list_train_clf = [f.split('.h5')[0] for f in os.listdir('i_vectors/feat/train_clf')]
ubm_list_validation = [f.split('.h5')[0] for f in os.listdir('i_vectors/feat/validation')]
ubm_list_test = sorted([f.split('.h5')[0] for f in os.listdir('i_vectors/feat/test')])

### Now we are ready to train UBM model using data provided in train_ivectors folder.

In [11]:
ubm = sidekit.Mixture()
ubm.EM_split(features_server=fs_train_ivectors,
             feature_list=ubm_list_train_ivectors,
             distrib_nb=distribNb,
             save_partial=True,
             iterations=(1, 2, 2, 4, 4, 4, 4, 8, 8, 8, 8, 8, 8),
             num_thread=nbThread,
             ceil_cov=10,
             floor_cov=1e-2)

ubm.write('i_vectors/ubm/ubm_{}.h5'.format(distribNb))

### Now we use trained UBM model to enroll all the sufficient statistics in order to train T-matrix and finally extract i-vectors. 

In [17]:
ubm = sidekit.Mixture('i_vectors/ubm/ubm_{}.h5'.format(distribNb))

leftids_train_ivectors = np.array(['eng' if f.startswith('jurcic') else 'oth' for f in ubm_list_train_ivectors])
rightids_train_ivectors = np.array(ubm_list_train_ivectors)

leftids_train_clf = np.array(['eng' if f.startswith('jurcic') else 'oth' for f in ubm_list_train_clf])
rightids_train_clf = np.array(ubm_list_train_clf)

leftids_validation = np.array(['eng' if f.startswith('jurcic') else 'oth' for f in ubm_list_validation])
rightids_validation = np.array(ubm_list_validation)

leftids_test = np.array(['eng', 'eng', 'eng', 'oth', 'eng', 'oth', 'oth'])
rightids_test = np.array(ubm_list_test)

def get_idmap(leftids, rightids):
    
    idmap = sidekit.IdMap()
    idmap.leftids = leftids
    idmap.rightids = rightids
    idmap.start = np.empty((len(leftids)), dtype="|O")
    idmap.stop = np.empty((len(leftids)), dtype="|O")
    
    return idmap

idmap_train_ivectors = get_idmap(leftids_train_ivectors, rightids_train_ivectors)
idmap_train_clf = get_idmap(leftids_train_clf, rightids_train_clf)
idmap_validation = get_idmap(leftids_validation, rightids_validation)
idmap_test = get_idmap(leftids_test, rightids_test)


def save_stat(idmap, fs, ubm, distrib_num, thread_num, save_path):
    
    enroll_stat = sidekit.StatServer(idmap,
                                     distrib_nb=distrib_num,
                                     feature_size=39)

    enroll_stat.accumulate_stat(ubm=ubm, 
                                feature_server=fs,
                                seg_indices=range(enroll_stat.segset.shape[0]),
                                num_thread=thread_num)

    enroll_stat.write(save_path)
    
save_stat(idmap_train_ivectors, fs_train_ivectors, ubm, distribNb, nbThread, 'i_vectors/statisics_train_ivectors.h5')
save_stat(idmap_train_clf, fs_train_clf, ubm, distribNb, nbThread, 'i_vectors/statisics_train_clf.h5')
save_stat(idmap_validation, fs_validation, ubm, distribNb, nbThread, 'i_vectors/statisics_validation.h5')
save_stat(idmap_test, fs_test, ubm, distribNb, nbThread, 'i_vectors/statisics_test.h5')

### Now let's train and save total variability matrix.

In [107]:
fa = sidekit.FactorAnalyser()

res = fa.total_variability('i_vectors/statisics_train_ivectors.h5',
                           ubm,
                           rank_TV,
                           nb_iter=10,
                           min_div=True,
                           tv_init=None,
                           batch_size=6400,
                           save_init=False,
                           output_file_name='i_vectors/total_variability_train',
                           num_thread=nbThread)

### And finally extract i-vectors to fit the classifier. 

In [87]:
fa = sidekit.FactorAnalyser('i_vectors/total_variability_train.h5')

def extract_ivectors(factor_analyser, ubm, stat_path, num_thread, batch_size):
    
    ivectors = factor_analyser.extract_ivectors(ubm,
                                                stat_path,
                                                prefix='',
                                                batch_size=batch_size,
                                                uncertainty=False,
                                                num_thread=num_thread)
    
    return ivectors


ivectors_test = extract_ivectors(fa, ubm, 'i_vectors/statisics_test.h5', nbThread, 100)
ivectors_validation = extract_ivectors(fa, ubm, 'i_vectors/statisics_validation.h5', nbThread, 10700)
ivectors_train_clf = extract_ivectors(fa, ubm, 'i_vectors/statisics_train_clf.h5', nbThread, 10700)

### Now let's fit XGBoost

In [204]:
X = ivectors_train_clf.stat1
y = [0 if i == b'eng' else 1 for i in ivectors_train_clf.modelset]

X_val = ivectors_validation.stat1
y_val = [0 if i == b'eng' else 1 for i in ivectors_validation.modelset]

X_test = ivectors_test.stat1

d_train = xgb.DMatrix(X, y)
d_val = xgb.DMatrix(X_val, y_val)

params = dict(learning_rate=0.1,
              n_estimators=1000,
              max_depth=6,
              min_child_weight=1,
              objective='binary:logistic',
              gamma=0,
              nthread=0,
              max_delta_step=1,
              subsample=1.0,
              colsample_bytree=1.0,
              colsample_bylevel=1.0,
              eval_metric='auc',
              seed=1234)

model = xgb.train(params, d_train, num_boost_round=300, evals=[(d_train,'train'), (d_val, 'val')])
preds_test_xgb = model.predict(xgb.DMatrix(ivectors_test.stat1))
preds_val_xgb = model.predict(d_val)

print('AUC ROC on validation set: {}'.format(roc_auc_score(y_val, preds_val_xgb)))

[0]	train-auc:0.777415	val-auc:0.703077
[1]	train-auc:0.831411	val-auc:0.747498
[2]	train-auc:0.860363	val-auc:0.770572
[3]	train-auc:0.892588	val-auc:0.79543
[4]	train-auc:0.910096	val-auc:0.810513
[5]	train-auc:0.927646	val-auc:0.824696
[6]	train-auc:0.936872	val-auc:0.833528
[7]	train-auc:0.945016	val-auc:0.843326
[8]	train-auc:0.953112	val-auc:0.855153
[9]	train-auc:0.960998	val-auc:0.864413
[10]	train-auc:0.966411	val-auc:0.870413
[11]	train-auc:0.971636	val-auc:0.876678
[12]	train-auc:0.975794	val-auc:0.883361
[13]	train-auc:0.977964	val-auc:0.887133
[14]	train-auc:0.980679	val-auc:0.892658
[15]	train-auc:0.983487	val-auc:0.897589
[16]	train-auc:0.985067	val-auc:0.900049
[17]	train-auc:0.987252	val-auc:0.904802
[18]	train-auc:0.989086	val-auc:0.9086
[19]	train-auc:0.989955	val-auc:0.910486
[20]	train-auc:0.991281	val-auc:0.913759
[21]	train-auc:0.99219	val-auc:0.916853
[22]	train-auc:0.993186	val-auc:0.918692
[23]	train-auc:0.99401	val-auc:0.921191
[24]	train-auc:0.994711	val-auc

[221]	train-auc:1	val-auc:0.983315
[222]	train-auc:1	val-auc:0.983413
[223]	train-auc:1	val-auc:0.983424
[224]	train-auc:1	val-auc:0.983481
[225]	train-auc:1	val-auc:0.983532
[226]	train-auc:1	val-auc:0.983558
[227]	train-auc:1	val-auc:0.98356
[228]	train-auc:1	val-auc:0.983606
[229]	train-auc:1	val-auc:0.983624
[230]	train-auc:1	val-auc:0.98364
[231]	train-auc:1	val-auc:0.983664
[232]	train-auc:1	val-auc:0.983713
[233]	train-auc:1	val-auc:0.983755
[234]	train-auc:1	val-auc:0.983755
[235]	train-auc:1	val-auc:0.983796
[236]	train-auc:1	val-auc:0.983825
[237]	train-auc:1	val-auc:0.9839
[238]	train-auc:1	val-auc:0.983926
[239]	train-auc:1	val-auc:0.983925
[240]	train-auc:1	val-auc:0.98395
[241]	train-auc:1	val-auc:0.983982
[242]	train-auc:1	val-auc:0.984057
[243]	train-auc:1	val-auc:0.984035
[244]	train-auc:1	val-auc:0.98408
[245]	train-auc:1	val-auc:0.984104
[246]	train-auc:1	val-auc:0.984141
[247]	train-auc:1	val-auc:0.984145
[248]	train-auc:1	val-auc:0.984185
[249]	train-auc:1	val-auc:

### Now let's figure out the probability threshold according to validation set using F1 metric

In [205]:
pr, rec, thr = precision_recall_curve(y_val, preds_val_xgb)

f1 = []
for p, r, t in zip(pr, rec, thr):
    f1.append(2*p*r/(p+r))

print('Precision at max F1: {0}; Recall at max F1: {1}; Threshold at max F1: {2}'.format(round(pr[np.argmax(f1)], 3),
                                                                                         round(rec[np.argmax(f1)], 3),
                                                                                         round(thr[np.argmax(f1)], 3)))

Precision at max F1: 0.949; Recall at max F1: 0.956; Threshold at max F1: 0.6190000176429749


### Now let's check results on the test set

In [220]:
for pred, true in zip(preds_test_xgb, ivectors_test.modelset):
    print('Probability: {:.2f}; Real flag: {}'.format(pred, true))

Probability: 0.48; Real flag: b'eng'
Probability: 1.00; Real flag: b'eng'
Probability: 0.44; Real flag: b'eng'
Probability: 0.85; Real flag: b'oth'
Probability: 0.29; Real flag: b'eng'
Probability: 1.00; Real flag: b'oth'
Probability: 0.92; Real flag: b'oth'


## conclusions

It seems that my models make some mistakes on the first wav file of the test set provided:
    
    1 part of en-en.wav file must have the probability lower 0.619 and got 0.48
    2 part of en-en.wav file must have the probability lower 0.619 but got 1.00
    3 part of en-en.wav file must have the probability lower 0.619 and got 0.44
    
But model manage to produce correct probabilities for fr-en.wav and it-tr.wav files
    
    1 part of fr-en.wav file must have the probability higher 0.619 and got 0.85
    2 part of fr-en.wav file must have the probability lower 0.619 and got 0.29
    
    1 part of it-tr.wav file must have the probability higher 0.619 and got 1.00
    2 part of it-tr.wav file must have the probability higher 0.619 and got 0.92

# EXTRA TEST

I was also provided with extra test set of audios, so lets extract i-vectors to be able to see how good xgboost model performes.

In [169]:
data_path_test2 = 'lid_test_small/'
save_path_test2 = 'i_vectors/test2/'

files_test2 = os.listdir(data_path_test2)
for f in files_test2:    
    convert(os.path.join(data_path_test2, f), save_path_test2)
    

extract_folder = save_path_test2
extract_folder_list = extract_folder.split('/')
extract_folder_list.insert(1, 'feat')
feat_folder = '/'.join(extract_folder_list)

extractor = sidekit.FeaturesExtractor(audio_filename_structure=os.path.join(extract_folder, '{}.wav'),
                                      feature_filename_structure=os.path.join(feat_folder, '{}.h5'),
                                      sampling_frequency=8000,
                                      lower_frequency=200,
                                      higher_frequency=3700,
                                      filter_bank="log",
                                      filter_bank_size=24,
                                      window_size=0.025,
                                      shift=0.01,
                                      ceps_number=20,
                                      vad="snr",
                                      snr=40,
                                      pre_emphasis=0.97,
                                      save_param=["vad", "energy", "cep", "fb"],
                                      keep_all_features=True)

filenames = [f.split('.')[0] for f in os.listdir(extract_folder)]
channel_list = np.zeros(len(filenames), dtype=np.int8)
extractor.save_list(show_list=filenames,
                    channel_list=channel_list,
                    num_thread=nbThread)

fs_test2 = get_feature_server('i_vectors/feat/test2/{}.h5')
ubm_list_test2 = [f.split('.h5')[0] for f in os.listdir('i_vectors/feat/test2')]

ubm = sidekit.Mixture('i_vectors/ubm/ubm_{}.h5'.format(distribNb))

leftids_test2 = np.array(['eng' if f.startswith('english') else 'oth' for f in ubm_list_test2])
rightids_test2 = np.array(ubm_list_test2)

idmap_test2 = get_idmap(leftids_test2, rightids_test2)
save_stat(idmap_test2, fs_test2, ubm, distribNb, nbThread, 'i_vectors/statisics_test2.h5')

fa = sidekit.FactorAnalyser('i_vectors/total_variability_train.h5')
ivectors_test2 = extract_ivectors(fa, ubm, 'i_vectors/statisics_test2.h5', nbThread, 100)

## Calculate ROC AUC, Precision, Recall and F1 score.

In [223]:
print('ROC AUC: {:.2f}'.format(roc_auc_score(y_test2, preds_test2_xgb)))
binary_preds = [0 if pred < 0.6190000176429749 else 1 for pred in preds_test2_xgb]

print('F1: {:.2f}; Precision: {:.2f}; Recall: {:.2f}'.format(f1_score(y_test2, binary_preds),
                                                             precision_score(y_test2, binary_preds),
                                                             recall_score(y_test2, binary_preds)))

ROC AUC: 0.99
F1: 0.76; Precision: 0.61; Recall: 1.00
